In [80]:
import os
from dotenv import load_dotenv
load_dotenv()
# Check if variables are loaded
groq_key = os.getenv('GROQ_API_KEY')
langchain_key = os.getenv('LANGCHAIN_API_KEY')
if not groq_key or not langchain_key:
    raise ValueError("Missing required environment variables in .env file")
os.environ['GROQ_API_KEY'] = groq_key
os.environ['LANGCHAIN_API_KEY'] = langchain_key

In [81]:
from langchain_groq import ChatGroq
LLM=ChatGroq(model='llama-3.1-8b-instant',temperature=0.7, max_tokens=512)

In [82]:
#Simple Genai Application
from langchain_community.document_loaders import WebBaseLoader

In [83]:
Webdata=WebBaseLoader("https://docs.langchain.com/oss/python/langchain/overview")
docs = Webdata.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/oss/python/langchain/overview', 'title': 'LangChain overview - Docs by LangChain', 'description': 'LangChain is an open source framework with a pre-built agent architecture and integrations for any model or tool — so you can build agents that adapt as fast as the ecosystem evolves', 'language': 'en'}, page_content='LangChain overview - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...⌘KSupportGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestA

In [84]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [85]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/oss/python/langchain/overview', 'title': 'LangChain overview - Docs by LangChain', 'description': 'LangChain is an open source framework with a pre-built agent architecture and integrations for any model or tool — so you can build agents that adapt as fast as the ecosystem evolves', 'language': 'en'}, page_content='LangChain overview - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...⌘KSupportGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestA

In [86]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [87]:
from langchain_community.vectorstores import Chroma
db=Chroma.from_documents(documents, embeddings)

In [88]:
db.similarity_search("What is LangChain?", k=2)

[Document(metadata={'source': 'https://docs.langchain.com/oss/python/langchain/overview', 'language': 'en', 'title': 'LangChain overview - Docs by LangChain', 'description': 'LangChain is an open source framework with a pre-built agent architecture and integrations for any model or tool — so you can build agents that adapt as fast as the ecosystem evolves'}, page_content='LangChain overview - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...⌘KSupportGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestA

In [89]:
#query from vector store db
query="Explain Artificial Intelligence in simple terms"
result=db.similarity_search(query)
result[0].page_content

'The Turing test can provide some evidence of intelligence, but it penalizes non-human intelligent behavior.[401]\nRussell and Norvig agree with Turing that intelligence must be defined in terms of external behavior, not internal structure.[1] However, they are critical that the test requires the machine to imitate humans. "Aeronautical engineering texts", they wrote, "do not define the goal of their field as making \'machines that fly so exactly like pigeons that they can fool other pigeons.\'"[402] AI founder John McCarthy agreed, writing that "Artificial intelligence is not, by definition, simulation of human intelligence".[403]\nMcCarthy defines intelligence as "the computational part of the ability to achieve goals in the world".[404] Another AI founder, Marvin Minsky, similarly describes it as "the ability to solve hard problems".[405] The leading AI textbook defines it as the study of agents that perceive their environment and take actions that maximize their chances of achievin

In [90]:
#Retrival chain, Document chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """Answer the following question based on the provided context.
    If you don't know the answer, just say that you don't know.
    <context>
    {context}
    </context>
    """
)

document_chain=create_stuff_documents_chain(LLM,prompt=prompt) #Create docuemnt chain where we pass multiple document
#We combine all document DOC1+DOC2+DOC3+LLM-->Answer

In [ ]:
#Document must come from retrival
retriever=db.as_retriever(search_kwargs={"k": 3})
from langchain_classic.chains.retrieval import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [92]:
response=retrieval_chain.invoke({"input": "Explain langchain ?"})
response['answer']

'Based on the provided context, I will answer your question to the best of my abilities.\n\nWhat is LangChain and its benefits?\n\nLangChain is an open-source framework with a pre-built agent architecture and integrations for any model or tool, allowing you to build agents that adapt as fast as the ecosystem evolves. It provides a standard model interface, making it easy to swap providers and avoid lock-in. LangChain is the easiest way to start building agents and applications powered by LLMs, with under 10 lines of code.'

In [94]:
response 

{'input': 'Explain langchain ?',
 'context': [Document(metadata={'description': 'LangChain is an open source framework with a pre-built agent architecture and integrations for any model or tool — so you can build agents that adapt as fast as the ecosystem evolves', 'source': 'https://docs.langchain.com/oss/python/langchain/overview', 'title': 'LangChain overview - Docs by LangChain', 'language': 'en'}, page_content='LangChain overview - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...⌘KSupportGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term